## Etapa 1

- Identificar os conjuntos de dados de criminalidade disponíveis na Secretaria de Segurança
Pública.

- Baixar todos os datasets relevantes (ex.: diferentes séries de procedência, diferentes anos
ou categorias).

- Organizar os arquivos em pastas, nomeando de forma clara.


In [ ]:
import pandas as pd
import numpy as np
# Carregando os datasets
df_2021 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2021.csv", sep=";", encoding="latin1", low_memory=False)
df_2022 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2022.csv", sep=";", encoding="latin1", low_memory=False)
df_2023 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2023.csv", sep=";", encoding="latin1", low_memory=False)
df_2024 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2024.csv", sep=";", encoding="latin1", low_memory=False)
df_2025 = pd.read_csv("./Dados Criminais 2021 - 2025/dados_2025.csv", sep=";", encoding="latin1", low_memory=False)

print("Datasets carregados:")
print("2021 ->", df_2021.shape, "linhas e colunas")
print("2022 ->", df_2022.shape, "linhas e colunas")
print("2023 ->", df_2023.shape, "linhas e colunas")
print("2024 ->", df_2024.shape, "linhas e colunas")
print("2025 ->", df_2025.shape, "linhas e colunas")

df_2022.head()


## Etapa 2

- Importar os arquivos em Python (pandas).
- Garantir que as colunas estejam padronizadas (nomes iguais, tipos de dados coerentes).
- Concatenar todos os dados de criminalidade em um único DataFrame.


In [ ]:
# Concatenar em um único dataframe
df_crimes = pd.concat([df_2021, df_2022, df_2023, df_2024, df_2025], ignore_index=True)
print("Dataframe consolidado:", df_crimes.shape)
df_crimes.head()


In [ ]:
print("CHECKPOINT ETAPA 1 - COLETA DE DADOS")
print(f"Total de registros consolidados: {df_crimes.shape[0]:,}")
print(f"Total de colunas: {df_crimes.shape[1]}")
print("Arquivos coletados: dados_2021.csv, dados_2022.csv, dados_2023.csv, dados_2024.csv, dados_2025.csv")


In [ ]:
# Remover colunas 'unnamed' geradas por excesso de separadores ou células vazias
df_crimes = df_crimes.loc[:, ~df_crimes.columns.str.lower().str.startswith('unnamed')]

In [ ]:
# Renomear colunas principais
df_crimes.rename(columns={
    "Sequência": "sequencia",
    "Data Fato": "data_fato", 
    "Hora Fato": "hora_fato",
    "Grupo Fato": "grupo_fato",
    "Tipo Enquadramento": "tipo_enquadramento",
    "Tipo Fato": "tipo_fato",
    "Municipio Fato": "municipio_fato",
    "Local Fato": "local_fato",
    "Bairro": "bairro",
    "Quantidade Vítimas": "quantidade_vitimas",
    "Idade Vítima": "idade_vitima",
    "Sexo Vítima": "sexo_vitma",
    "Cor Vítima": "cor_vitma",
}, inplace=True)
df_crimes


## Etapa 3
- Tratar valores ausentes (decidir se preenche, exclui ou substitui).
- Remover duplicatas.
- Ajustar o formato de datas para um padrão único (YYYY-MM-DD).
- Corrigir inconsistências em colunas categóricas (ex.: "Homicídio" vs "homicidio").


In [ ]:
# Filtrar apenas os dados de Passo Fundo
df_pf = df_crimes[df_crimes['municipio_fato'].str.upper() == 'PASSO FUNDO']
df_pf.head()

In [ ]:
# Remover a última coluna (vazia) dados vazios
df_pf = df_pf.iloc[:, :-1]  
print(f"Coluna vazia removida. Novo shape: {df_pf.shape}")

In [ ]:
import unicodedata

# Normalizar texto (minúsculas, sem acentos, sem espaços)
df_pf = df_pf.applymap(
    lambda x: ''.join(
        c for c in unicodedata.normalize('NFKD', str(x).strip().lower()) 
        if not unicodedata.combining(c)
    ) if isinstance(x, str) else x
)
print("=== DADOS DE CRIMINALIDADE LIMPOS E PADRONIZADOS ===")
print(f"Shape: {df_pf.shape}")

In [ ]:
# Limpeza e padronização dos dados

# 1. Remover duplicatas
df_pf = df_pf.drop_duplicates()

# 2. Substituir "Sem informação" por NaN
df_pf.replace("sem informacao", np.nan, inplace=True)

# 3. Preencher colunas numéricas com a média
numericas = df_pf.select_dtypes(include=['int64', 'float64']).columns
df_pf[numericas] = df_pf[numericas].fillna(df_pf[numericas].mean())

# 4. Converter datas e horas
df_pf['data_fato'] = pd.to_datetime(df_pf['data_fato'], format='%d/%m/%Y', errors='coerce')
df_pf['hora_fato'] = pd.to_datetime(df_pf['hora_fato'], format='%H:%M:%S', errors='coerce').dt.time

# 5. Garantir que numéricas permaneçam numéricas
df_pf[numericas] = df_pf[numericas].apply(pd.to_numeric, errors='coerce')

# 6 Categóricas -> categoria Ignorado
df_pf['sexo_vitma'] = df_pf['sexo_vitma'].fillna("ignorado")
df_pf['cor_vitma']  = df_pf['cor_vitma'].fillna("ignorado")




In [ ]:
df_pf.head()

In [ ]:
df_pf.info()

In [ ]:
# --- Bloco de Correção de Tipos de Dados ---

# 1. Converter colunas numéricas principais
df_pf['sequencia'] = pd.to_numeric(df_pf['sequencia'], errors='coerce')
df_pf['quantidade_vitimas'] = pd.to_numeric(df_pf['quantidade_vitimas'], errors='coerce')
df_pf['idade_vitima'] = pd.to_numeric(df_pf['idade_vitima'], errors='coerce')

# 2. Preencher nulos apenas onde faz sentido
# Sequência e quantidade de vítimas não devem ficar nulos
df_pf[['sequencia', 'quantidade_vitimas']] = df_pf[['sequencia', 'quantidade_vitimas']].fillna(0)

# 3. Ajustar tipos definitivos
df_pf['sequencia'] = df_pf['sequencia'].astype(int)
df_pf['quantidade_vitimas'] = df_pf['quantidade_vitimas'].astype(int)
df_pf['idade_vitima'] = df_pf['idade_vitima'].round().astype('Int64')

# 4. Converter hora_fato para datetime.time
# Substituir strings inválidas por NaN
df_pf['hora_fato'] = df_pf['hora_fato'].replace(
    ['ignorado', 'sem informação', 'nao informado'], np.nan
)

# Tentar converter para datetime.time
df_pf['hora_fato'] = pd.to_datetime(df_pf['hora_fato'], format='%H:%M:%S', errors='coerce').dt.time

# Conferir resultados
print(df_pf['hora_fato'].unique()[:50])

# 5. Converter colunas categóricas para economizar memória
cat_cols = [
    'grupo_fato', 'tipo_enquadramento', 'tipo_fato',
    'municipio_fato', 'local_fato', 'bairro',
    'sexo_vitma', 'cor_vitma'
]
df_pf[cat_cols] = df_pf[cat_cols].astype('category')

# --- Verificação Final ---
print("Tipos de dados após a correção:")
df_pf.info()
print("\nExemplo de linhas:")
print(df_pf.head())


## Etapa 4 

- Importar o dataset meteorológico (fornecido pelo professor).
- Garantir que a coluna de data esteja no mesmo formato que o dataset de criminalidade.
- Fazer o merge dos datasets (chave: data + cidade = Passo Fundo).
- Garantir que nenhuma coluna gerou dados vazios. Caso alguma informação esteja vazia,
- preencher com alguma informação consistente.


In [ ]:
import pandas as pd

caminho_meteo = r"C:\Users\Eduardo\Documents\Codes\dados-criminalidade-meteorologia\Dados meteo\passo_fundo_meteriologia.csv"

# Ler o CSV pulando as linhas de metadados
df_meteo = pd.read_csv(
    caminho_meteo, 
    sep=';', 
    decimal=',',          # para interpretar vírgula como decimal
    skiprows=9,           # pula as 9 primeiras linhas de metadados
    encoding='latin1'
)

# Conferir as primeiras linhas
print(df_meteo.head())


In [ ]:
df_meteo.head()

In [ ]:
df_meteo.shape

In [ ]:
# Remover colunas 'unnamed' vazias
df_meteo = df_meteo.drop(columns=['Unnamed: 6'])


In [ ]:
# Renomear a coluna de data
df_meteo.rename(columns={'Data Medicao': 'data'}, inplace=True)

# Converter para datetime e padronizar formato YYYY-MM-DD (igual df_pf)
df_meteo['data'] = pd.to_datetime(df_meteo['data'], errors='coerce').dt.strftime('%Y-%m-%d')


In [ ]:
# 1. Garantir que a coluna de data está em datetime
df_pf['data_fato'] = pd.to_datetime(df_pf['data_fato'], errors='coerce')
df_meteo['data'] = pd.to_datetime(df_meteo['data'], errors='coerce')

# 2. Merge usando data como chave
df_merged = pd.merge(
    df_pf, 
    df_meteo, 
    left_on='data_fato', 
    right_on='data', 
    how='inner'
)

# 3. Conferir resultado
print(f"Crimes: {df_pf.shape}, Meteorologia: {df_meteo.shape}, Merge: {df_merged.shape}")
df_merged.head()


In [ ]:
# Remover a coluna 'data' duplicada (mantendo 'data_fato')
df_merged = df_merged.drop('data', axis=1)

# Verificar o resultado final
print("=== DATASET INTEGRADO ===")
print(f"Shape final: {df_merged.shape}")
print(f"Período: {df_merged['data_fato'].min()} a {df_merged['data_fato'].max()}")
print("\nPrimeiras linhas:")
df_merged.head()

In [ ]:
# Preencher dados meteorológicos vazios com 0
df_merged['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)'].fillna(0, inplace=True)
df_merged['TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)'].fillna(20, inplace=True)  
df_merged['TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)'].fillna(10, inplace=True)
df_merged['UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)'].fillna(60, inplace=True)
df_merged['VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)'].fillna(5, inplace=True)

print("Dados preenchidos! Verificando...")
print("Dados vazios restantes:", df_merged.isnull().sum().sum())

In [88]:
# TESTE. Verificar dados vazios no dataset integrado
print("ver dados vazios")
print("Valores nulos por coluna:")
print(df_merged.isnull().sum())

print("\nPercentual de dados vazios:")
percentual_vazios = (df_merged.isnull().sum() / len(df_merged)) * 100
print(percentual_vazios[percentual_vazios > 0])

ver dados vazios
Valores nulos por coluna:
sequencia                                           0
data_fato                                           0
hora_fato                                           0
grupo_fato                                          0
tipo_enquadramento                                  0
tipo_fato                                           0
municipio_fato                                      0
local_fato                                          0
bairro                                           6547
quantidade_vitimas                                  0
idade_vitima                                        0
sexo_vitma                                          0
cor_vitma                                           0
PRECIPITACAO TOTAL, DIARIO (AUT)(mm)                0
TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)               0
TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)               0
UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)       0
VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/

In [ ]:
# TESTE. Ver informações básicas do dataset final
print("=== INFORMAÇÕES DO DATASET FINAL ===")
print(f"Total de registros: {len(df_merged):,}")
print(f"Total de colunas: {df_merged.shape[1]}")
print(f"Período: {df_merged['data_fato'].min()} até {df_merged['data_fato'].max()}")
print("\nPrimeiras 5 linhas:")
df_merged.head()

=== INFORMAÇÕES DO DATASET FINAL ===
Total de registros: 59,124
Total de colunas: 18
Período: 2021-10-01 00:00:00 até 2025-08-31 00:00:00

Primeiras 5 linhas:


,sequencia,data_fato,hora_fato,grupo_fato,tipo_enquadramento,tipo_fato,municipio_fato,local_fato,bairro,quantidade_vitimas,idade_vitima,sexo_vitma,cor_vitma,"PRECIPITACAO TOTAL, DIARIO (AUT)(mm)","TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)","TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)","UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)","VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)"
0,7,2021-10-01,00:01:00,crimes,furto abigeato,consumado,passo fundo,outros,rod transbrasiliana,1,58,masculino,branca,10.6,23.0,15.6,91.6,3.1
1,65,2021-10-01,01:10:00,crimes,entorpecentes - trafico,consumado,passo fundo,residencia,vila popular,0,40,ignorado,ignorado,10.6,23.0,15.6,91.6,3.1
2,71,2021-10-01,01:19:00,contravencoes,vias de fato,consumado,passo fundo,residencia,rodriguez,1,48,feminino,branca,10.6,23.0,15.6,91.6,3.1
3,162,2021-10-01,03:30:09,crimes,furto de documento,consumado,passo fundo,outros,centro,1,20,feminino,ignorado,10.6,23.0,15.6,91.6,3.1
4,188,2021-10-01,04:14:00,contravencoes,perturbacao do trabalho ou do sossego alheios,consumado,passo fundo,residencia,vila planaltina,0,40,ignorado,ignorado,10.6,23.0,15.6,91.6,3.1


In [89]:
# TESTE. Verificar se os dados fazem sentido
print("=== VERIFICAÇÃO DE RELEVÂNCIA ===")
print("Tipos de crimes mais comuns:")
print(df_merged['tipo_fato'].value_counts().head())

print("\nEstatísticas das variáveis meteorológicas:")
colunas_meteo = ['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)', 
                'TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)', 
                'TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)']
print(df_merged[colunas_meteo].describe())

=== VERIFICAÇÃO DE RELEVÂNCIA ===
Tipos de crimes mais comuns:
tipo_fato
consumado    59124
Name: count, dtype: int64

Estatísticas das variáveis meteorológicas:
       PRECIPITACAO TOTAL, DIARIO (AUT)(mm)  \
count                          59124.000000   
mean                               5.316531   
std                               13.855323   
min                                0.000000   
25%                                0.000000   
50%                                0.000000   
75%                                2.600000   
max                              153.600000   

       TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)  \
count                           59124.000000   
mean                               24.755265   
std                                 5.586633   
min                                 6.000000   
25%                                20.900000   
50%                                25.100000   
75%                                29.200000   
max                           

In [90]:
# TESTE. Teste simples de correlação
print("=== TESTE DE RELEVÂNCIA ===")
print("Crimes por dia em média:", len(df_merged) / df_merged['data_fato'].nunique())
print("Dias únicos com dados:", df_merged['data_fato'].nunique())

=== TESTE DE RELEVÂNCIA ===
Crimes por dia em média: 41.316561844863735
Dias únicos com dados: 1431


In [91]:
# TESTE. Ver se tem correlação básica
crimes_por_dia = df_merged.groupby('data_fato').size()
print(f"Dia com mais crimes: {crimes_por_dia.max()} crimes")
print(f"Dia com menos crimes: {crimes_por_dia.min()} crimes")

Dia com mais crimes: 79 crimes
Dia com menos crimes: 14 crimes


#### possiveis preocupações


bairros vazios, ver como tratar



In [92]:
# Filtra o DataFrame para mostrar apenas as linhas onde 'bairro' é nulo
linhas_vazias = df_merged[df_merged['bairro'].isnull()]

# Exibe as primeiras 10 linhas desse filtro
print("Exibindo as primeiras 10 linhas onde o bairro é nulo:")
display(linhas_vazias.head(10))

Exibindo as primeiras 10 linhas onde o bairro é nulo:


,sequencia,data_fato,hora_fato,grupo_fato,tipo_enquadramento,tipo_fato,municipio_fato,local_fato,bairro,quantidade_vitimas,idade_vitima,sexo_vitma,cor_vitma,"PRECIPITACAO TOTAL, DIARIO (AUT)(mm)","TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)","TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)","UMIDADE RELATIVA DO AR, MEDIA DIARIA (AUT)(%)","VENTO, VELOCIDADE MEDIA DIARIA (AUT)(m/s)"
12,513,2021-10-01,09:00:00,crimes,estelionato,consumado,passo fundo,outros,NaN,1,25,feminino,branca,10.6,23.0,15.6,91.6,3.1
20,880,2021-10-01,10:50:00,crimes,outras fraudes,consumado,passo fundo,outros,NaN,0,40,ignorado,ignorado,10.6,23.0,15.6,91.6,3.1
21,895,2021-10-01,11:00:00,crimes,exercicio arbitrario proprias razoes,consumado,passo fundo,hospitais/clinicas,NaN,1,53,feminino,branca,10.6,23.0,15.6,91.6,3.1
30,1426,2021-10-01,15:00:00,crimes,difamacao,consumado,passo fundo,outros,NaN,1,57,feminino,branca,10.6,23.0,15.6,91.6,3.1
32,1587,2021-10-01,15:45:00,crimes,violar a suspensao ou proibicao de dirigir vei...,consumado,passo fundo,via publica,NaN,0,40,ignorado,ignorado,10.6,23.0,15.6,91.6,3.1
35,1695,2021-10-01,16:30:00,crimes,"permitir,confiar,ou entreg. direcao a pessoa s...",consumado,passo fundo,via publica,NaN,0,40,ignorado,ignorado,10.6,23.0,15.6,91.6,3.1
58,2804,2021-10-02,04:00:00,crimes,furto em veiculo,consumado,passo fundo,via publica,NaN,1,19,masculino,branca,0.0,24.0,13.3,83.2,2.8
72,3502,2021-10-02,14:00:00,crimes,lesao corporal,consumado,passo fundo,outros,NaN,1,14,feminino,branca,0.0,24.0,13.3,83.2,2.8
84,4015,2021-10-02,18:56:00,crimes,lesao corporal,consumado,passo fundo,via publica,NaN,1,22,masculino,preta,0.0,24.0,13.3,83.2,2.8
89,4246,2021-10-02,21:00:00,crimes,lesao corporal,consumado,passo fundo,via publica,NaN,1,32,feminino,branca,0.0,24.0,13.3,83.2,2.8


Para verificar ainda e testar

# Verificar dados vazios no dataset integrado
print("ver dados vazios")
print("Valores nulos por coluna:")
print(df_merged.isnull().sum())

print("\nPercentual de dados vazios:")
percentual_vazios = (df_merged.isnull().sum() / len(df_merged)) * 100
print(percentual_vazios[percentual_vazios > 0])


# 2. Ver informações básicas do dataset final
print("=== INFORMAÇÕES DO DATASET FINAL ===")
print(f"Total de registros: {len(df_merged):,}")
print(f"Total de colunas: {df_merged.shape[1]}")
print(f"Período: {df_merged['data_fato'].min()} até {df_merged['data_fato'].max()}")
print("\nPrimeiras 5 linhas:")
df_merged.head()

# 3. Verificar se os dados fazem sentido
print("=== VERIFICAÇÃO DE RELEVÂNCIA ===")
print("Tipos de crimes mais comuns:")
print(df_merged['tipo_fato'].value_counts().head())

print("\nEstatísticas das variáveis meteorológicas:")
colunas_meteo = ['PRECIPITACAO TOTAL, DIARIO (AUT)(mm)', 
                'TEMPERATURA MAXIMA, DIARIA (AUT)(Â°C)', 
                'TEMPERATURA MINIMA, DIARIA (AUT)(Â°C)']
print(df_merged[colunas_meteo].describe())

# 4. Teste simples de correlação
print("=== TESTE DE RELEVÂNCIA ===")
print("Crimes por dia em média:", len(df_merged) / df_merged['data_fato'].nunique())
print("Dias únicos com dados:", df_merged['data_fato'].nunique())

# Ver se tem correlação básica
crimes_por_dia = df_merged.groupby('data_fato').size()
print(f"Dia com mais crimes: {crimes_por_dia.max()} crimes")
print(f"Dia com menos crimes: {crimes_por_dia.min()} crimes")